In [14]:
import random
import mysql.connector
import json
import string
import datetime as dt
from pymongo import MongoClient
from pprint import pprint

def getGenero(argument):
    switcher = {        
        5: "action",
        6: "adventure",
        7: "casual",
        8: "strategy",
        9: "RPG",
        10: "simulation",
        11: "early_access",
        12: "free_to_play",
        13: "sports",
        14: "racing",
        15: "massively_multiplayer"
    }
    return switcher.get(argument, argument)

myclient = MongoClient("mongodb://localhost:27017/")
mydbmongo = myclient['stum_for_you']
mycol = mydbmongo["usuarios"]
colmul = mydbmongo["multidimensional"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)
from datetime import datetime,timedelta
hoy = datetime.now()
dias = timedelta(days=7)
semana_pasada = hoy-dias  
#se comprueba que sea domingo (0)
comp=hoy.strftime("%w")
print(comp)
if (comp == '0') :#--------------------------------------------------------------cuidado
    
    hoy=hoy.strftime("%Y-%m-%d")
    #semana_pasada=semana_pasada.strftime("%Y:%m:%d")
    
    semana_pasada ="2017-01-01"#-------------------------------------------------para pruebas
    print(hoy,semana_pasada)
    mycursor = mydb.cursor()

    mycursor.execute("SELECT * FROM transacciones_venta WHERE fecha_venta BETWEEN %s AND  %s", (semana_pasada,hoy,))
    transacciones = mycursor.fetchall()
    tipo_movimiento = "venta"
    
    for t in transacciones :
        es_juego = False
        es_dlc = False
        existe_descuento = False
        existe_descuento_personalizado = False
   
    #"movimiento"
        precio_final = t[1]
        print(t[0])
        mycursor.execute("SELECT * FROM venta_juegos WHERE id_transaccion = %s", (t[0],))
        venta = mycursor.fetchall()
        cantidad = len(venta)
    #fecha
        anyo=t[2].strftime("%Y")
        mes=t[2].strftime("%m")
        dia=t[2].strftime("%d")
    #juego
        if(len(venta)>0) :
            es_juego = True
            mycursor.execute("SELECT id_juego FROM claves_juegos WHERE id_clave = %s", (venta[0][1],))
            puntero_juego = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM juegos WHERE id_juego = %s", (puntero_juego,))
            juego = mycursor.fetchone()
            genero= []
            #sacar genero
            for a in range (5,15) :
                if (juego[a]==True) :
                    genero.append(getGenero(a))
            precio_juego=juego[3]
            clasificacion=juego[1]
            #descuento_personalizado
            puntero_usuario = venta[0][2]
            descuento_personalizado = mycol.find_one({"id_usuario": puntero_usuario, "$or" :[{"cupon":{"tipo": "all"}},{"cupon":{"tipo": "dlc"}}]})
            if descuento_personalizado:
                existe_descuento_personalizado = True
                porcentaje_personalizado = descuento_personalizado["porcentaje"]
                fecha_inicio_personalizado = descuento_personalizado["fecha_inicio"]
                fecha_fin_personalizado = descuento_personalizado["fecha_fin"]
                tipo_personalizado = descuento_personalizado["tipo"]
            
            #descuento
            mycursor.execute("SELECT d.* FROM descuentos d JOIN claves_juegos c ON c.id_juego = d.id_juego WHERE c.id_clave = %s", (venta[0][1],))
            descuento = mycursor.fetchone()
            if descuento:
                existe_descuento = True
                porcentaje = descuento[1]
                fecha_inicio = descuento[2]
                fecha_fin = descuento[3]

     #dlc
        if(len(venta)<=0) :
            es_dlc = True
            mycursor.execute("SELECT * FROM venta_dlcs WHERE id_transaccion = %s", (t[0],))
            venta = mycursor.fetchall()
            cantidad = len(venta)
            mycursor.execute("SELECT id_dlc FROM claves_dlc WHERE id_clave = %s", (venta[0][0],))
            puntero_dlc = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM dlcs WHERE id_dlc = %s", (puntero_dlc,))
            dlc = mycursor.fetchone()
            tipo= dlc[3]
            precio_dlc=dlc[1]
            #descuento_personalizado
            puntero_usuario = venta[0][2]
            descuento_personalizado = mycol.find_one({"id_usuario": puntero_usuario, "$or" :[{"cupon":{"tipo": "all"}},{"cupon":{"tipo": "dlc"}}]})
            if descuento_personalizado:
                existe_descuento_personalizado = True
                porcentaje_personalizado = descuento_personalizado["porcentaje"]
                fecha_inicio_personalizado = descuento_personalizado["fecha_inicio"]
                fecha_fin_personalizado = descuento_personalizado["fecha_fin"]
                tipo_personalizado = descuento_personalizado["tipo"]
            
            #descuento
            mycursor.execute("SELECT d.* FROM descuentos d JOIN claves_dlcs c ON c.id_dlc = d.id_dlc WHERE c.id_clave = %s", (venta[0][0],))
            descuento = mycursor.fetchone()
            if descuento:
                existe_descuento = True
                porcentaje = descuento[1]
                fecha_inicio = descuento[2]
                fecha_fin = descuento[3]
            
    #usuario
        puntero_usuario=venta[0][2]
        usuario = mycol.find_one({"id_usuario": puntero_usuario})
        sexo = usuario["sexo"]
        fechaNacimiento = usuario["fecha_nacimiento"]
        #ubicacion
        pais = usuario["pais"]
        ciudad = usuario["ciudad"]

        datos = {
            "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
            "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
            "ubicacion":{"pais":pais,"ciudad":ciudad},
            "usuario":{"sexo":sexo,"fechaNacimiento":fechaNacimiento,"pais":pais,"ciudad":ciudad},
            }
        if es_juego:
            datos["juego"] = {"puntero_juego":puntero_juego,"genero":genero,"precio":precio_juego,"clasificacion":clasificacion}
        if es_dlc:
            datos["dlc"] = {"precio":precio_dlc,"tipo":tipo}
        if existe_descuento:
            datos["descuento"] = {"porcentaje":porcentaje,"fecha_inicio":fecha_inicio,"fecha_fin":fecha_fin}
        if existe_descuento_personalizado:
            datos["descuento_personalizado"] = {"porcentaje":porcentaje_personalizado,"fecha_inicio":fecha_inicio_personalizado,"fecha_fin":fecha_fin_personalizado, "tipo":tipo_personalizado}
        
        print("---------------------------------------------------------------")
        pprint(datos)
    

0
2020-01-12 2017-01-01
1


ProgrammingError: Failed processing format-parameters; Python 'tuple' cannot be converted to a MySQL type